# Predicting the stock market


In this project, I'll work with data from the S&P500 Index. The S&P500 is a stock [market index](https://en.wikipedia.org/wiki/S%26P_500). 

I'll be using historical data on the price of the S&P500 Index to make predictions about future prices. Predicting whether an index goes up or down helps forecast how the stock market as a whole performs. Since stocks tend to correlate with how well the economy as a whole is performs, it can also help with economic forecasts. 

The columns of the dataset are:

- Date -- The date of the record.
- Open -- The opening price of the day (when trading starts).
- High -- The highest trade price during the day.
- Low -- The lowest trade price during the day.
- Close -- The closing price for the day (when trading is finished).
- Volume -- The number of shares traded.
- Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions. Read more [here](https://www.investopedia.com/terms/a/adjusted_closing_price.asp).

I will be working with a csv file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in sphist.csv.

I'll train the model with data from 1950-2012 and try to make predictions from 2013-2015.

## Reading in the Data

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from datetime import datetime

pd.options.display.max_columns = 999

In [2]:
df = pd.read_csv('~/Documents/Python/sphist.csv')
df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883
5,2015-11-30,2090.949951,2093.810059,2080.409912,2080.409912,4.245030e+09,2080.409912
6,2015-11-27,2088.820068,2093.290039,2084.129883,2090.110107,1.466840e+09,2090.110107
7,2015-11-25,2089.300049,2093.000000,2086.300049,2088.870117,2.852940e+09,2088.870117
8,2015-11-24,2084.419922,2094.120117,2070.290039,2089.139893,3.884930e+09,2089.139893
9,2015-11-23,2089.409912,2095.610107,2081.389893,2086.590088,3.587980e+09,2086.590088


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       16590 non-null  object 
 1   Open       16590 non-null  float64
 2   High       16590 non-null  float64
 3   Low        16590 non-null  float64
 4   Close      16590 non-null  float64
 5   Volume     16590 non-null  float64
 6   Adj Close  16590 non-null  float64
dtypes: float64(6), object(1)
memory usage: 907.4+ KB


In [4]:
# Convert the Date column to a Pandas date type

df['Date'] = pd.to_datetime(df['Date'])

In [5]:
# generate a Boolean series that tells us if each item in the Date column is after 2015-04-01

df["Date"] > datetime(year=2015, month=4, day=1)

0         True
1         True
2         True
3         True
4         True
         ...  
16585    False
16586    False
16587    False
16588    False
16589    False
Name: Date, Length: 16590, dtype: bool

In [6]:
# sort the dataframe on the Date column

df = df.sort_values(by='Date', ascending = True)
df.head(10)


,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999


## Generating Indicators


In [7]:
#Calculate average price from the past 5, 30, 365 days
df['day_5'] = df['Close'].rolling(5).mean().shift(1)
df['day_30'] = df['Close'].rolling(30).mean().shift(1)
df['day_365'] = df['Close'].rolling(365).mean().shift(1)

#Calculate the STD for the past 5, 365 days
df['std_5'] = df['Close'].rolling(5).std().shift(1)
df['std_365'] = df['Close'].rolling(365).std().shift(1)


In [8]:
df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_30,day_365,std_5,std_365
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,NaN,NaN,0.157956,NaN
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,NaN,NaN,0.089051,NaN
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,NaN,NaN,0.067602,NaN
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,NaN,NaN,0.134796,NaN
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,NaN,NaN,0.196545,NaN


In [9]:
df.tail(10)

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_30,day_365,std_5,std_365
9,2015-11-23,2089.409912,2095.610107,2081.389893,2086.590088,3.587980e+09,2086.590088,2071.523974,2061.892989,2033.605890,18.246940,64.911334
8,2015-11-24,2084.419922,2094.120117,2070.290039,2089.139893,3.884930e+09,2089.139893,2078.204004,2064.197327,2034.018028,15.807754,64.768328
7,2015-11-25,2089.300049,2093.000000,2086.300049,2088.870117,2.852940e+09,2088.870117,2085.943994,2067.045658,2034.432712,3.491188,64.634873
6,2015-11-27,2088.820068,2093.290039,2084.129883,2090.110107,1.466840e+09,2090.110107,2087.002002,2070.199996,2034.835123,3.395982,64.514871
5,2015-11-30,2090.949951,2093.810059,2080.409912,2080.409912,4.245030e+09,2080.409912,2088.776025,2072.408333,2035.199864,1.309055,64.449800
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2087.024023,2073.984998,2035.531178,3.916109,64.370261
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2090.231982,2076.283993,2035.914082,7.956808,64.352527
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2088.306006,2077.908659,2036.234356,9.333599,64.277554
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.456006,2078.931331,2036.507343,19.599946,64.121622
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068,2080.771973,2080.237329,2036.869425,19.806136,64.058862


Since I am computing indicators that use historical data, there are some rows where there isn't enough historical data to generate them. There is an indicator that uses 365 days of historical data, and the dataset starts on 1950-01-03. So I have to remove the data before 1951-01-03.

In [10]:
#Remove the data before 1951-01-03
df = df[df["Date"] > datetime(year=1951, month=1, day=2)]

# Drop all rows containing null values
df_clean = df.dropna(axis = 0)
df_clean.isnull().sum()


Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
day_5        0
day_30       0
day_365      0
std_5        0
std_365      0
dtype: int64

## Making Predictions


In [11]:

def train_test(df, features):
    
    train = df[df['Date'] < datetime(year=2013, month=1, day=1)]
    test = df[df['Date'] >= datetime(year=2013, month=1, day=1)]

    target = 'Close'
    
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse


In [12]:
# examine the function with 'day_5', 'day_30', 'day_365' columns

features = ['day_5', 'day_30', 'day_365']
rmse = train_test(df_clean, features)
rmse

22.22006532421962

In [13]:
# examine the function with 'std_5', 'std_365' columns

features_1 = ['std_5', 'std_365']
rmse_1 = train_test(df_clean, features_1)
rmse_1

802.6816054338967

I'm going to add 5 additional indicators to dataframe and see if the error is reduced


In [14]:
#Calculate the mean volume for the past 5, 365 days
df['day_5_volume'] = df['Volume'].rolling(5).mean().shift(1)
df['day_365_volume'] = df['Volume'].rolling(365).mean().shift(1)

#Calculate the STD of the average volume over the past 5, 365 days 
df['5_volume_std'] = df['day_5_volume'].rolling(5).std().shift(1)
df['365_volume_std'] = df['day_365_volume'].rolling(365).std().shift(1)

#Calculate the ratio between the standard deviation of the average volume for the past five days, 
#and the standard deviation of the average volume for the past year.

df['ratio_5_365'] = df['std_5'] / df ['std_365']

In [15]:
# Drop all rows containing null values

df_new = df.dropna(axis = 0)
df_new.isnull().sum()


Date              0
Open              0
High              0
Low               0
Close             0
Volume            0
Adj Close         0
day_5             0
day_30            0
day_365           0
std_5             0
std_365           0
day_5_volume      0
day_365_volume    0
5_volume_std      0
365_volume_std    0
ratio_5_365       0
dtype: int64

In [16]:
# apply the function with 'day_5_volume', 'day_5_volume' columns

features = ['day_5_volume', 'day_365_volume']
rmse = train_test(df_new, features)
print('RMSE for features: mean_volume_5, mean_volume_365 is', rmse)


RMSE for features: mean_volume_5, mean_volume_365 is 732.6106802070286


In [17]:
# apply the function with '5_volume_std', '365_volume_std' columns

features = ['5_volume_std', '365_volume_std']
rmse = train_test(df_new, features)
print('RMSE for features: 5_volume_std, 365_volume_std is', rmse)


RMSE for features: 5_volume_std, 365_volume_std is 1235.1059550358518


In [18]:
# apply the function with 'ratio_5_365' columns

features = ['ratio_5_365']
rmse = train_test(df_new, features)
print('RMSE for features: ratio_5_365 is', rmse)


RMSE for features: ratio_5_365 is 1445.6351772836933


In [19]:
# apply the function with all new columns

features = ['day_5', 'day_30', 'day_365', 'std_5', 'std_365','day_5_volume', 'day_365_volume', '5_volume_std', '365_volume_std', 'ratio_5_365']
rmse = train_test(df_new, features)
print('RMSE for features: all new columns is', rmse)


RMSE for features: all new columns is 22.203524321596888


As I see to use all new columns make the model with the lowest error.


## Make a preditions one day ahead

Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what I'd do if I were trading using the algorithm.

So let's do this type of algorithm


In [21]:
def train_test(df, features, year, month, day):
    
    train = df[df['Date'] < datetime(year=year, month=month, day=day)]
    test = df[df['Date'] == datetime(year=year, month=month, day=day)]

    target = 'Close'
    
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

In [22]:
# predictions for 2013-01-03

features = ['day_5', 'day_30', 'day_365', 'std_5', 'std_365','day_5_volume', 'day_365_volume', '5_volume_std', '365_volume_std', 'ratio_5_365']
rmse = train_test(df_new, features, year=2013, month=1, day=3)
print('RMSE for date 2013-01-03 for features: all new columns is', rmse)


RMSE for date 2013-01-03 for features: all new columns is 33.35760037538739


In [23]:
# predictions for 2013-01-04

features = ['day_5', 'day_30', 'day_365', 'std_5', 'std_365','day_5_volume', 'day_365_volume', '5_volume_std', '365_volume_std', 'ratio_5_365']
rmse = train_test(df_new, features, year=2013, month=1, day=4)
print('RMSE for date 2013-01-04 for features: all new columns is', rmse)


RMSE for date 2013-01-04 for features: all new columns is 32.01443694210093


As I see accuracy would improve greatly by making predictions only one day ahead.


## Conclusion

In this project, I used S&P500 Index data from 1950 to 2015 to make predictions of the close day price of the index. I practiced using the linear regression model with different indicators that I have added to the data set to make better predictions. I also tried making predictions only one day ahead to improve the accuracy of the predictions and it significantly reduces the error.
